In [1]:
import cadquery as cq
from jupyter_cadquery.cadquery import show
from jupyter_cadquery import set_defaults, set_sidecar

set_defaults(display='sidecar', axes=False, axes0=True, grid=True,
             default_color='lightgrey', tools=True, ortho=True)

from cq_gears import (SpurGear, RingGear, HerringboneGear,
                      HerringboneRingGear, SpurGear, BevelGear, BevelGearPair,
                      PlanetaryGearset, HerringbonePlanetaryGearset)

set_sidecar('Gears')

import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import datetime
from collections import Counter, defaultdict
import os

from report import Report

%matplotlib widget

Overwriting auto display for cadquery Workplane and Shape


In [2]:
rep = Report.gather_reports('./reports')[0]

In [3]:
fig, ax = plt.subplots()

startangle = (n_negatives / (n_negatives + n_positives)) * 180.0
fcolors = ['#%x44' % (14 - c * 2) for c in range(len(fails))]
flabels = ['',] + list(fails.keys())

ax.pie([n_positives,] + list(fails.values()),
       radius=1.4,
       colors=['white',] + fcolors,
       wedgeprops={'width' : 0.6, 'edgecolor' : 'w'},
       startangle=startangle,
       labels=flabels)

ax.pie((n_positives, n_negatives), radius=0.8, colors=('green', 'red'),
       wedgeprops={'edgecolor' : 'w'},
       labels=('passed', 'failed'),
       explode=(0.0, 0.0),
       startangle=startangle,
       labeldistance=0.4)

handles, labels = ax.get_legend_handles_labels()
handles = handles[-2:] + handles[:-2]
labels = labels[-2:] + labels[:-2]


ax.set(aspect="equal", title='Test results')
ax.legend(handles, labels, loc=(-0.3, 0.0))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
params = rep.fails['TestBevelGear']['BUILDING'][7].copy()
print(params)

# BevelGear.curve_points = 20
# BevelGear.surface_splines = 12
# BevelGear.spline_approx_tol = 0.01
# BevelGear.spline_approx_max_deg = 8

# params['cone_angle'] = 15.0
# params['face_width'] = 190.0
# params['helix_angle'] = 36.0

gear = BevelGear(**params)


# faces = gear._build_tooth_faces()
# faces = gear._build_gear_faces()

# wp = cq.Workplane('XY').add(faces)
solid = gear.build()
wp = cq.Workplane('XY').add(solid)

show(wp)

{'module': 3.905409871350235, 'teeth_number': 92, 'cone_angle': 20.03324219673066, 'face_width': 508.65385509902353, 'pressure_angle': 14.662362218239014, 'helix_angle': 67.20177029229427, 'trim_top': False, 'trim_bottom': False}


AssertionError: Twist angle is NaN

In [31]:
bb = wp.val().BoundingBox()
width = gear.cone_h - \
                np.cos(gear.gamma_f) * (gear.gs_r - gear.face_width)
print(width, bb.zmin, bb.zmax)
print(wp.val().Volume())

420.9937458388607 -0.5409198903116639 421.5346657291724
41238971.83162469


In [16]:
gear.twist_angle

nan

In [21]:
rp = gear.m * gear.z / 2.0
beta = np.arctan(gear.face_width * np.tan(gear.helix_angle) / (2.0 * gear.gs_r - gear.face_width))
twist_angle = np.arcsin(gear.gs_r / rp * np.sin(beta)) * 2.0

gear.gs_r / rp * np.sin(beta)

/tmp/ipykernel_874300/3329249642.py:3: RuntimeWarning: invalid value encountered in arcsin
  twist_angle = np.arcsin(gear.gs_r / rp * np.sin(beta)) * 2.0


1.1806721232780275

In [54]:
os.path.isfile

<function genericpath.isfile(path)>